In [1]:
import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook_connected"

import pandas as pd
import pycountry
from pycountry_convert import country_alpha2_to_continent_code, convert_continent_code_to_continent_name

# Function to get the continent of each country
def iso_to_continent(country_iso):
    try:
        country = pycountry.countries.get(alpha_3=country_iso)
        country_alpha2 = country.alpha_2
        continent_code = country_alpha2_to_continent_code(country_alpha2)
        continent_name = convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except (KeyError, AttributeError):
        return None

# Clean dataset    
df = pd.read_csv("population1.csv")
df = df.dropna(subset=['Country of origin (ISO)', 'Country of asylum (ISO)'])
df['asy_continent'] = df['Country of asylum (ISO)'].apply(iso_to_continent)
df = df.dropna()
df2 =df[df["Country of origin (ISO)"]=="UKR"]
df2 = df2[df2["Year"] >= 2014]

# Replace minor countries by other
for index, row in df2.iterrows():
    if row["Refugees under UNHCR's mandate"] < 108008 and row['asy_continent'] == "Europe":
        df2.loc[index, "Country of asylum"] = "Other European Countries"
    elif row["Refugees under UNHCR's mandate"] < 108008 and row['asy_continent'] != "Europe":
        df2.loc[index, "Country of asylum"] = "Other Countries"
        df2.loc[index, "asy_continent"] = "Other"

# Sum total refugees for each country
df3 = df2.groupby(["Country of asylum", "asy_continent"]).sum("Refugees under UNHCR's mandate").sort_values(by=["Refugees under UNHCR's mandate"], ascending=False)
df4 = df3[df3["Refugees under UNHCR's mandate"] != 0].reset_index()

# Set parents, labels, and values for the chart
parents = df4["asy_continent"].tolist()
labels = df4["Country of asylum"].tolist()
values = df4["Refugees under UNHCR's mandate"].tolist()

df = pd.DataFrame({'Labels': labels, 'Parents': parents, 'Values': values})

add_parents = ["Ukraine"]*2+[""]
add_labels = ["Europe", "Other", "Ukraine"]
add_values = [6513730, 77700, 6591430]
na_values = df["Values"].to_list()
na_labels = df["Labels"].to_list()
na_Parents = df["Parents"].to_list()

# Plot by plotly
import plotly.graph_objects as go

fig =go.Figure(go.Sunburst(
    labels=add_labels+na_labels,
    parents=add_parents+na_Parents,
    values=add_values+na_values,
    branchvalues="total",
))

fig.update_layout(
    title=dict(
    text="Asylum Countries of Ukraine Refugees from 2014 to 2022",
    font=dict(
    family="Arial", size=20)
    ),

    yaxis_title=dict(
    # text="Asylum Countries of Ukraine Refugees from 2014 to 2022",
    font=dict(
    family="Arial", size=18)
    ),

    xaxis_title=dict(
    # text="Asylum Countries of Ukraine Refugees from 2014 to 2022",
    font=dict(
    family="Arial", size=18)
    )
)

fig.show()